# 2. 유전 알고리즘 기본 원리

이 노트북에서는 유전 알고리즘의 핵심 개념과 구현을 단계별로 학습합니다.

## 목차
1. [유전 알고리즘이란?](#1-유전-알고리즘이란)
2. [진화의 핵심 요소들](#2-진화의-핵심-요소들)
3. [간단한 최적화 문제로 실습](#3-간단한-최적화-문제로-실습)
4. [신경망과 유전 알고리즘](#4-신경망과-유전-알고리즘)
5. [파라미터 영향 분석](#5-파라미터-영향-분석)

---


## 1. 유전 알고리즘이란?

### 생물학적 진화를 모방한 최적화 알고리즘

**다윈의 진화론 핵심:**
- 변이(Variation): 개체들은 서로 다른 특성을 가짐
- 유전(Inheritance): 부모의 특성이 자손에게 전달됨  
- 선택(Selection): 환경에 적합한 개체가 생존하고 번식
- 시간(Time): 여러 세대에 걸쳐 점진적 개선

### 유전 알고리즘의 기본 구조

```
1. 초기 개체군 생성 (무작위)
2. 적합도 평가 (각 개체의 성능 측정)
3. 선택 (좋은 개체들을 부모로 선택)
4. 교차 (부모들의 특성을 조합하여 자손 생성)
5. 돌연변이 (무작위 변화로 다양성 확보)
6. 2-5 반복 (종료 조건까지)
```

### 왜 유전 알고리즘을 사용하는가?
- **전역 최적해 탐색**: 지역 최적해에 빠지지 않음
- **복잡한 문제 해결**: 수학적 해법이 없는 문제도 해결
- **병렬 탐색**: 여러 해답을 동시에 탐색
- **강건성**: 노이즈나 불완전한 정보에도 잘 작동


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Tuple

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (10, 6)

print("라이브러리 로드 완료!")


## 2. 진화의 핵심 요소들

각 진화 요소를 간단한 예제로 이해해보겠습니다.


In [ ]:
class SimpleGeneticAlgorithm:
    """간단한 유전 알고리즘 구현 (실수 최적화)"""
    
    def __init__(self, population_size=20, gene_length=10, mutation_rate=0.1):
        self.population_size = population_size
        self.gene_length = gene_length
        self.mutation_rate = mutation_rate
        
        # 개체군 초기화 (각 개체는 실수 배열)
        self.population = []
        for _ in range(population_size):
            individual = np.random.uniform(-5, 5, gene_length)
            self.population.append(individual)
        
        self.generation = 0
        self.best_fitness_history = []
        self.avg_fitness_history = []
    
    def fitness_function(self, individual):
        """적합도 함수 - 간단한 함수 최적화 문제"""
        # 목표: x^2 + y^2 + ... 를 최소화 (0에 가까워야 함)
        return -np.sum(individual ** 2)  # 음수로 만들어서 최대화 문제로 변환
    
    def evaluate_population(self):
        """모든 개체의 적합도 평가"""
        fitness_scores = []
        for individual in self.population:
            fitness = self.fitness_function(individual)
            fitness_scores.append(fitness)
        return np.array(fitness_scores)
    
    def selection(self, fitness_scores, num_parents=10):
        """토너먼트 선택"""
        parents = []
        for _ in range(num_parents):
            # 무작위로 3개 개체 선택하여 가장 좋은 것 선택
            candidates = random.sample(range(self.population_size), 3)
            winner_idx = max(candidates, key=lambda x: fitness_scores[x])
            parents.append(self.population[winner_idx].copy())
        return parents
    
    def crossover(self, parent1, parent2):
        """단순 교차 (한 점 교차)"""
        crossover_point = random.randint(1, self.gene_length - 1)
        
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        
        return child1, child2
    
    def mutation(self, individual):
        """가우시안 돌연변이"""
        mutated = individual.copy()
        for i in range(len(individual)):
            if random.random() < self.mutation_rate:
                mutated[i] += np.random.normal(0, 0.5)  # 가우시간 노이즈 추가
                mutated[i] = np.clip(mutated[i], -10, 10)  # 범위 제한
        return mutated
    
    def evolve_one_generation(self):
        """한 세대 진화"""
        # 1. 적합도 평가
        fitness_scores = self.evaluate_population()
        
        # 2. 통계 기록
        best_fitness = np.max(fitness_scores)
        avg_fitness = np.mean(fitness_scores)
        self.best_fitness_history.append(best_fitness)
        self.avg_fitness_history.append(avg_fitness)
        
        # 3. 부모 선택
        parents = self.selection(fitness_scores, num_parents=10)
        
        # 4. 새로운 개체군 생성
        new_population = []
        
        # 엘리트 보존 (최고 개체 몇 개 유지)
        elite_indices = np.argsort(fitness_scores)[-2:]  # 상위 2개
        for idx in elite_indices:
            new_population.append(self.population[idx].copy())
        
        # 교차와 돌연변이로 나머지 개체 생성
        while len(new_population) < self.population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = self.crossover(parent1, parent2)
            
            child1 = self.mutation(child1)
            child2 = self.mutation(child2)
            
            new_population.extend([child1, child2])
        
        # 개체군 크기 맞추기
        self.population = new_population[:self.population_size]
        self.generation += 1
        
        return best_fitness, avg_fitness

# 유전 알고리즘 인스턴스 생성
ga = SimpleGeneticAlgorithm(population_size=30, gene_length=5, mutation_rate=0.2)
print(f"초기 개체군 크기: {len(ga.population)}")
print(f"각 개체의 유전자 길이: {ga.gene_length}")

# 초기 적합도 확인
initial_fitness = ga.evaluate_population()
print(f"초기 최고 적합도: {np.max(initial_fitness):.3f}")
print(f"초기 평균 적합도: {np.mean(initial_fitness):.3f}")


In [ ]:
# 진화 과정 실행
print("진화 시작!")
for generation in range(50):
    best_fit, avg_fit = ga.evolve_one_generation()
    
    if generation % 10 == 0 or generation == 49:
        print(f"세대 {generation}: 최고={best_fit:.3f}, 평균={avg_fit:.3f}")

# 결과 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(ga.best_fitness_history, 'r-', label='최고 적합도', linewidth=2)
plt.plot(ga.avg_fitness_history, 'b--', label='평균 적합도', linewidth=2)
plt.xlabel('세대')
plt.ylabel('적합도')
plt.title('진화 과정')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
final_fitness = ga.evaluate_population()
plt.hist(final_fitness, bins=10, alpha=0.7, color='green')
plt.xlabel('적합도')
plt.ylabel('개체 수')
plt.title('최종 개체군의 적합도 분포')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"\\n최종 결과:")
print(f"최고 적합도: {np.max(final_fitness):.6f}")
print(f"최고 개체: {ga.population[np.argmax(final_fitness)]}")
